In [158]:
%matplotlib inline
#import dateutil.parser
#import datetime
#import sqlalchemy
import urllib2
#import xml
#import xml.etree.ElementTree
from lxml import etree
#from lxml import objectify
#from bs4 import BeautifulSoup
#import metadataDB
import metadataDB.addArticle
import metadataDB.getmetadata
import metadataDB.harvest
import itertools
import time
#import pyoai

In [166]:
reload(metadataDB.harvest)
metadataDB.harvest.harvest()

Total time: 10.414547 for 131 entries (0.079500 per entry)


In [170]:
print "%f hours" % (0.08 * 1e6/(3600.))

22.222222 hours


In [156]:
url = 'http://export.arxiv.org/oai2?verb=ListRecords&set=physics&from=2015-01-01&until=2015-02-01&metadataPrefix=arXiv'
#url = 'http://export.arxiv.org/oai2?verb=ListRecords&set=physics:cond-mat&from=2015-01-01&until=2015-02-01&metadataPrefix=arXiv'
reload(metadataDB.getmetadata)
#url = metadataDB.getmetadata.constructURL()
#print url
start = time.time()
xmlList = metadataDB.getmetadata.download(url)
final = time.time()
print final - start
# 0.06 sec per entry

xml is long... at 0 / 3643.
xml is long... at 1000 / 3643.
xml is long... at 2000 / 3643.
220.557451963


In [157]:
reload(metadataDB.getmetadata)
listArticle = list(itertools.chain.from_iterable([metadataDB.getmetadata.parse(x) for x in xmlList]))
reload(metadataDB.addArticle)
print len(listArticle)
start = time.time()
for article in listArticle:
    metadataDB.addArticle.addArticle(article)
final = time.time()
print 'Total time: %f for %i entries (%f per entry)' % (final - start, len(listArticle), (final-start)/len(listArticle))
# 0.025 seconds per entry
# For 1 million entries, that's 7 hours!

3643
Total time: 92.325708 for 3643 entries (0.025343 per entry)


The OAI-PMH interface

Remember to handle XML vulnerabilities!

https://docs.python.org/2/library/xml.html#xml-vulnerabilities

In [5]:
# A single record
'http://export.arxiv.org/oai2?verb=GetRecord&identifier=oai:arXiv.org:0804.2273&metadataPrefix=arXiv'

'http://export.arxiv.org/oai2?verb=GetRecord&identifier=oai:arXiv.org:0804.2273&metadataPrefix=arXiv'

In [54]:
testurl = 'http://export.arxiv.org/oai2?verb=ListRecords&set=physics:cond-mat&from=2015-01-01&until=2015-02-01&metadataPrefix=arXiv'
#testurl = 'http://export.arxiv.org/oai2?verb=ListRecords&set=math&from=2015-01-01&until=2015-01-03&metadataPrefix=arXiv'
#opener = urllib2.build_opener()    
#opener.addheaders = [('User-agent', 'Mozilla/5.0')]
#try:
# response = opener.open(testurl)
# html = response.read()
# response.close()

In [55]:
response = opener.open(testurl)
# html = response.read()
# response.close()

In [56]:
html = response.read()
response.close()

In [60]:
print html[-500:-1]

ef><abstract>  We present a quantum solution to coordination problems that can be
implemented with present technologies. It provides an alternative to existing
approaches, which rely on explicit communication, prior commitment or trusted
third parties. This quantum mechanism applies to a variety of scenarios for
which existing approaches are not feasible.
</abstract></arXiv>
</metadata>
</record>
<resumptionToken cursor="1000" completeListSize="1161"></resumptionToken>
</ListRecords>
</OAI-PMH>


In [58]:
e = etree.fromstring(html)

In [69]:
f = e.find('.//{http://www.openarchives.org/OAI/2.0/}resumptionToken')
print f.tag
print f.text
print f.attrib
'HTML is long... cursor is at %s / %s.'% (f.attrib['cursor'], f.attrib['completeListSize'])

{http://www.openarchives.org/OAI/2.0/}resumptionToken
None
{'cursor': '1000', 'completeListSize': '1161'}


'HTML is long... cursor is at 1000 / 1161.'